In [1]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount = True)

Mounted at /content/gdrive/


In [ ]:
checkpoint_dir = "/content/gdrive/MyDrive/MATS/Mech_Interp/models/checkpoints"
checkpoint = "genial-firefly-18"

In [2]:
!git clone https://github.com/sbp354/mats_sae_training.git

Cloning into 'mats_sae_training'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 332 (delta 201), reused 214 (delta 87), pack-reused 0
Receiving objects: 100% (332/332), 35.22 MiB | 11.11 MiB/s, done.
Resolving deltas: 100% (201/201), done.


In [3]:
import sys
sys.path.append("/content/mats_sae_training")

In [4]:
!pip install -q transformer_lens
!pip install -q einops
!pip install -q jaxtyping

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [17]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

from transformer_lens import HookedTransformer, utils
import torch
from datasets import load_dataset
import time
import os
from typing import Optional, List, Dict, Callable, Tuple, Union
import tqdm.notebook as tqdm
from pathlib import Path
import pickle
import plotly.express as px
import importlib
import json
import einops
import gzip
import shutil

from sae_analysis.visualizer import data_fns, model_fns, html_fns
# from model_fns import AutoEncoderConfig, AutoEncoder
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
torch.set_grad_enabled(False)
device = 'cuda:0'

In [ ]:
import gzip
import shutil

for file in os.listdir(os.path.join(checkpoint_dir, checkpoint)):
    with gzip.open(os.path.join(checkpoint_dir, checkpoint, str(file)), 'rb') as f_in:
        with open(os.path.join(checkpoint_dir, checkpoint, str(file)[:-3]), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)



BadGzipFile: ignored

In [8]:

from sae_training.sparse_autoencoder import SparseAutoencoder

def load_sae(file_path):
    with gzip.open(os.path.join(checkpoint_dir, checkpoint, str(file_path)), 'rb') as f_in:
        with open(os.path.join(checkpoint_dir, checkpoint, str(file_path)[:-3]), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    file = open(os.path.join(checkpoint_dir, checkpoint, str(file_path)[:-3]), 'rb')

    sae_data = pickle.load(file)
    file.close()

    sae = SparseAutoencoder(sae_data['cfg'])

    sae.load_from_pretrained(file_path)

    return sae

In [9]:
model = HookedTransformer.from_pretrained("gpt2-small").to(device)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda:0


In [10]:
data = load_dataset("stas/openwebtext-10k", split = 'train')
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=128)
tokenized_data = tokenized_data.shuffle(22)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


/usr/local/lib/python3.10/dist-packages/datasets/download/download_manager.py:524: FutureWarning: 'num_proc' was deprecated in version 2.6.2 and will be removed in 3.0.0. Pass `DownloadConfig(num_proc=<num_proc>)` to the initializer instead.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (55582 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (56043 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (56550 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (59222 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (60902 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence

In [ ]:
sae.W_dec.shape

torch.Size([98304, 768])

In [ ]:
px.line(sae.W_dec[0].cpu())

In [ ]:
px.line(sae.W_dec[-1].cpu())

In [ ]:
import einops
head_weights = einops.rearrange(sae.W_dec, "d_sae (n_head d_head) -> d_sae n_head d_head", n_head = 12, d_head = 64)

In [11]:
def calculate_skewness(data, dim=1):
    # Step 1: Calculate the mean and standard deviation along the specified dimension
    mean = torch.mean(data, dim=dim, keepdim=True)
    std_dev = torch.std(data, dim=dim, keepdim=True)

    # Step 2 & 3: Center and standardize the data
    standardized_data = (data - mean) / std_dev

    # Step 4: Cube the standardized data
    cubed_data = torch.pow(standardized_data, 3)

    # Step 5: Calculate the mean of the cubed data along the specified dimension
    skewness = torch.mean(cubed_data, dim=dim)

    return skewness

In [ ]:
skew = calculate_skewness(torch.sum(torch.abs(head_weights), -1))
skew.shape

torch.Size([98304])

In [ ]:
val_max, arg_max = torch.max(torch.abs(skew), dim =0)
print(val_max, arg_max)

tensor(2.3646, device='cuda:0') tensor(84639, device='cuda:0')


In [ ]:
px.line(sae.W_dec[arg_max].cpu())

In [ ]:
val_min, arg_min = torch.min(torch.abs(skew), dim =0)
print(val_min, arg_min)

tensor(8.9407e-08, device='cuda:0') tensor(89395, device='cuda:0')


In [ ]:
px.line(sae.W_dec[arg_min].cpu())

In [ ]:
px.line(torch.sum(torch.abs(head_weights), -1).cpu()[arg_min])

In [ ]:
checkpoint = 'treasured-star-17'
sae_treasured_start = load_sae(os.path.join(checkpoint_dir, checkpoint, 'final_sparse_autoencoder_gpt2-small_blocks.8.attn.hook_z_49152.pkl.gz'))

In [ ]:
head_weights = einops.rearrange(sae_treasured_start.W_dec, "d_sae (n_head d_head) -> d_sae n_head d_head", n_head = 12, d_head = 64)

In [ ]:
skew = calculate_skewness(torch.sum(torch.abs(head_weights), -1))
val_max, arg_max = torch.max(torch.abs(skew), dim =0)
print(val_max, arg_max)

tensor(2.4368, device='cuda:0') tensor(16673, device='cuda:0')


In [ ]:
px.line(sae_treasured_start.W_dec[arg_max].cpu())

In [ ]:
px.histogram((torch.sum(torch.abs(sae_treasured_start.W_dec < 0.01), dim = 1)/ sae_treasured_start.W_dec.shape[1]).cpu())

In [ ]:
print([i.item() for i in torch.argwhere(torch.sum(torch.abs(sae_treasured_start.W_dec < 0.01), dim = 1)/ sae_treasured_start.W_dec.shape[1]>0.64)])

[4589, 11200, 12439, 16789, 22368, 24247, 36882, 38696, 40582, 42016, 43537, 44717, 45390, 45508, 46005, 47883]


In [ ]:
px.line(sae_treasured_start.W_dec[4589].cpu())

In [ ]:
importlib.reload(data_fns)
importlib.reload(html_fns)
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

In [ ]:
all_tokens = tokenized_data["tokens"]

In [ ]:
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

In [ ]:
from sae_analysis.visualizer import data_fns, model_fns, html_fns
max_batch_size = 512
total_batch_size = 4096*6
tokens = all_tokens[:total_batch_size]
feature_data: Dict[int, FeatureData] = get_feature_data(
    encoder=sae_treasured_start,
    # encoder_B=sparse_autoencoder,
    model=model,
    hook_point=sae_treasured_start.cfg.hook_point,
    hook_point_layer=sae_treasured_start.cfg.hook_point_layer,
    tokens=tokens,
    feature_idx=[4589, 11200, 12439, 16789, 22368, 24247, 36882, 38696, 40582, 42016, 43537, 44717, 45390, 45508, 46005, 47883],
    max_batch_size=max_batch_size,
    left_hand_k = 3,
    buffer = (5, 5),
    n_groups = 10,
    first_group_size = 20,
    other_groups_size = 5,
    verbose = True,
)

Storing model activations:   0%|          | 0/48 [00:00<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
sae_treasured_start.cfg.hook_point

'blocks.8.attn.hook_z'

## Gelu-2L

In [13]:
checkpoint_dir = "/content/gdrive/MyDrive/MATS/Mech_Interp/models/gelu_checkpoints"
checkpoint = "peach-planet-14"

In [18]:
sae_14 = load_sae(os.path.join(checkpoint_dir, checkpoint, 'final_sparse_autoencoder_gelu-2l_blocks.1.attn.hook_z_8192.pkl.gz'))

RuntimeError: ignored